## Backtesting result of 007 pytorch model

In [1]:
from backtesting import Backtest, Strategy
import pandas as pd

import datetime

/Users/kimfaicheang/workspace/snowball/venv/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
hist = pd.read_csv("hist007.csv")
# print(hist.shape)

df = pd.read_csv("df007.csv", parse_dates=['Datetime'])

df['u_sig'] = hist['0'] > 0.3
df['d_sig'] = hist['0'] < 0

df.set_index('Datetime', inplace=True)
df.index = df.index.tz_convert('Asia/Shanghai')

df

,Open,High,Low,Close,Adj Close,Volume,u_sig,d_sig
Datetime,,,,,,,,
2023-12-04 22:30:00+08:00,4564.370117,4566.580078,4558.390137,4566.580078,4566.580078,127454215,False,True
2023-12-04 22:35:00+08:00,4566.959961,4568.049805,4563.319824,4565.990234,4565.990234,88653884,False,True
2023-12-04 22:40:00+08:00,4566.009766,4569.209961,4565.479980,4567.479980,4567.479980,82133816,False,False
2023-12-04 22:45:00+08:00,4567.410156,4570.919922,4567.410156,4569.609863,4569.609863,87963286,False,True
2023-12-04 22:50:00+08:00,4569.680176,4571.839844,4567.049805,4569.140137,4569.140137,78119822,False,True
...,...,...,...,...,...,...,...,...
2024-02-23 00:55:00+08:00,5062.350098,5063.379883,5061.759766,5062.350098,5062.350098,18921857,NaN,NaN
2024-02-23 01:00:00+08:00,5062.299805,5065.020020,5059.620117,5064.839844,5064.839844,19641884,NaN,NaN
2024-02-23 01:05:00+08:00,5064.759766,5066.399902,5064.529785,5066.299805,5066.299805,16965845,NaN,NaN


In [3]:
class St(Strategy):
    def init(self):
        pass
        
    def next(self):
        if self.data['u_sig'][-1]: 
            self.buy()

        if self.position and self.data['d_sig'][-1]:
            self.position.close()

In [4]:
bt = Backtest(
    df,
    St,
    commission=.0001,
    trade_on_close=True,
    exclusive_orders=True 
)

stats = bt.run()

stats

Start                     2023-12-04 22:30...
End                       2024-02-23 01:15...
Duration                     80 days 02:45:00
Exposure Time [%]                   44.041451
Equity Final [$]                  9734.125468
Equity Peak [$]                  10063.972779
Return [%]                          -2.658745
Buy & Hold Return [%]                 10.8924
Return (Ann.) [%]                    -9.81796
Volatility (Ann.) [%]                2.886901
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.205789
Avg. Drawdown [%]                   -0.783757
Max. Drawdown Duration       70 days 02:35:00
Avg. Drawdown Duration       11 days 10:20:00
# Trades                                  951
Win Rate [%]                        46.687697
Best Trade [%]                       1.174971
Worst Trade [%]                     -0.632936
Avg. Trade [%]                    

In [5]:
class St3(Strategy):
    def init(self):
        self.start_time = datetime.time(22, 30, 0)
        self.end_time = datetime.time(2, 30, 0)
        pass
        
    def next(self):
        # 只在指定時間段交易
        t = self.data.index[-1].time()
        in_ts = t > self.start_time or t <= self.end_time

        if self.data['u_sig'][-1] and in_ts: 
            self.buy()

        if self.position and t > self.end_time:
            self.position.close()

        if self.position and self.data['d_sig'][-1]:
            self.position.close()

In [6]:
bt = Backtest(
    df,
    St3,
    commission=.0001,
    trade_on_close=True,
    exclusive_orders=True 
)

stats = bt.run()

stats

Start                     2023-12-04 22:30...
End                       2024-02-23 01:15...
Duration                     80 days 02:45:00
Exposure Time [%]                    27.01366
Equity Final [$]                  9598.338158
Equity Peak [$]                  10011.317986
Return [%]                          -4.016618
Buy & Hold Return [%]                 10.8924
Return (Ann.) [%]                  -14.426387
Volatility (Ann.) [%]                1.765915
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.151565
Avg. Drawdown [%]                   -2.146549
Max. Drawdown Duration       78 days 23:20:00
Avg. Drawdown Duration       40 days 00:03:00
# Trades                                  637
Win Rate [%]                        43.799058
Best Trade [%]                       0.235432
Worst Trade [%]                      -0.25121
Avg. Trade [%]                    